In [1]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

/home/indj/anaconda3/envs/voice/lib/python3.8/site-packages/transformers/configuration_utils.py:380: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [2]:
import numpy as np

In [6]:
from huggingface_hub import notebook_login

notebook_login()

In [7]:
# audio_dataset.ipynb 에서 만든 데이터셋 불러오기

import datasets

label_0_dataset = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/label_0_dataset_librosa")
label_1_dataset = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/label_1_dataset_librosa")

In [8]:
# model-2 : 추가데이터
angry_dataset = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/angry_dataset_hugging")
happy_dataset = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/HAPPY_dataset_hugging")
neutral_dataset = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/NEUTRAL_dataset_hugging")

ne_data = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/neutral_dataset_hugging_2")
an_data = datasets.load_from_disk("/mnt/c/Users/user/angry_level_classification/data/angry_dataset_hugging_2")

In [9]:
from datasets import Dataset, concatenate_datasets

minds_local = concatenate_datasets([label_0_dataset, label_1_dataset])
minds_local

Dataset({
    features: ['audio', 'intent_class'],
    num_rows: 5501
})

In [10]:
minds_hugging = concatenate_datasets([angry_dataset, happy_dataset, neutral_dataset, ne_data, an_data])
minds_hugging

Dataset({
    features: ['audio', 'intent_class', 'file', 'label'],
    num_rows: 78999
})

In [11]:
label2id = {'angry':'1','neutral':'0'}
id2label = {'1':'angry','0':'neutral'}
num_labels = len(id2label)

In [12]:
def preprocess_function_loacal(examples):
    audio_arrays = [np.array(x["array"]) for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [13]:
minds_local = minds_local.train_test_split(test_size=0.2)

In [14]:
encoded_minds_local = minds_local.map(preprocess_function_loacal, remove_columns="audio", batched=True)

Map:   0%|          | 0/4400 [00:00<?, ? examples/s]

Map:   0%|          | 0/1101 [00:00<?, ? examples/s]

In [15]:
encoded_minds_local['train'][0]['input_values'][0]

0.0002665127976797521

In [16]:
type(encoded_minds_local['train'][0]['input_values'][0])

float

In [17]:
def preprocess_function_hugging(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays, sampling_rate=feature_extractor.sampling_rate, max_length=16000, truncation=True
    )
    return inputs

In [18]:
minds_hugging = minds_hugging.train_test_split(test_size=0.2)

In [ ]:
encoded_minds_hugging = minds_hugging.map(preprocess_function_hugging, remove_columns="audio", batched=True)

Map:   0%|          | 0/63199 [00:00<?, ? examples/s]

In [ ]:
encoded_minds_hugging['train'][0]['input_values'][0]

In [ ]:
type(encoded_minds_hugging['train'][0]['input_values'][0])

In [ ]:
# type(np.array(minds_local['audio'][0]['array']))

In [ ]:
encoded_minds_hugging

In [ ]:
encoded_minds_local

In [ ]:
from datasets import DatasetDict, concatenate_datasets

# DatasetDict를 사용하여 두 데이터셋을 하나로 합칩니다.
encoded_minds = DatasetDict({
    "train": concatenate_datasets([encoded_minds_local["train"], encoded_minds_hugging["train"]]),
    "test": concatenate_datasets([encoded_minds_local["test"], encoded_minds_hugging["test"]])
})

In [ ]:
type(encoded_minds['train'][0]['input_values'][0])

In [ ]:
encoded_minds['train'][0]['input_values'][0]

In [ ]:
encoded_minds = encoded_minds.rename_column("intent_class", "label")

In [ ]:
encoded_minds

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=eval_pred.label_ids)

In [ ]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    "facebook/wav2vec2-base", num_labels=num_labels, label2id=label2id, id2label=id2label
)

In [ ]:
training_args = TrainingArguments(
    output_dir="my_voice_classification_model_2",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    # no_cuda =True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_minds["train"],
    eval_dataset=encoded_minds["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

trainer.train()

In [26]:
trainer.save_model("./model/my_voice_classification_model")

# 추론